In [168]:
# LLM 로드
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()  # VS Code에서 자동 로드 안 하면 필수

llm = ChatOpenAI(
    base_url=os.environ["OPENAI_BASE_URL"],   # http://172.16.10.168:9993/v1
    api_key=os.environ["OPENAI_API_KEY"],     # vLLM이면 더미도 OK
    model=os.environ["OPENAI_MODEL"],         # vLLM served model name과 정확히 일치
    temperature=0,
)

In [169]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
import pandas as pd
import re
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [170]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
# loader = WebBaseLoader(
#     web_paths=("http://www.apt-ami.co.kr/nuri/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             "div",
#             attrs={"class": "post-content"},
#         )
#     ),
# )

# docs = loader.load()
docs = pd.read_csv(r"C:\Users\nuri\Desktop\RAG\민생.csv", sep=None, engine="python", encoding="utf-8-sig")
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 9


,항목,내용
0,추진배경,민생경제 회복을 위한 조치로 소비활성화와 소상공인 및 자영업자 매출확대를 위해 민생...
1,대상,전 국민 대상으로 지급
2,지원금액,"1인당 15 ~ 55만원, 소득별 맞춤형 지원 및 단계적 지급"
3,지원방식,1차 : 전국민 1인당 15 ~ 40만원 우선 지급\r\n2차 : 전국민 90%에게...
4,신청 및 지급 기간,1차 : 2025.07.21.(월) ~ 09.12.(금)\r\n2차 : 2025.0...
5,신청방법,"온라인 : 카드사・지역사랑상품권 홈페이지, 앱, 콜센터, ARS\r\n오프라인 : ..."
6,사용처,지역사랑상품권 : 지역사랑상품권 가맹점\r\n신용・체크・선불카드 : 연 매출액 30...
7,사용기한,2025.11.30.(일)까지
8,기사,"구윤철 부총리 겸 기획재정부 장관이 3일 ""민생회복 소비쿠폰 등 정책 효과로 8월 ..."


In [171]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# splits = text_splitter.split_documents(docs)
# len(splits)

text = open("C:/Users/nuri/Desktop/RAG/민생.csv", encoding="utf-8-sig").read()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_text(text)
splits = [Document(page_content=c, metadata={"source": "민생.txt"}) for c in chunks]

In [172]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [173]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
    검색된 다음 문맥(context) 을 사용하여 질문(question)에 답하세요. 또한 사고과정(<think>)은 출력하지 말고 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 주어진 정보에서
    질문에 대한 정보를 찾을 수 없습니다 라고 답하세요. 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.



#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

In [174]:
# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def format_docs(docs):
    return "\n\n---\n\n".join(d.page_content for d in docs)

# think 블록 제거 (좀 더 관대한 패턴)
def drop_think(text: str) -> str:
    return re.sub(r"<\s*think\b[^>]*>.*?</\s*think\s*>\s*", "", text, flags=re.S|re.I)

In [177]:
# for chunk in rag_chain.stream("대한민국의 올해 실행된 정책중 민생지원금의 1차 지급과 관련해서 설명해주세요"):
#     print(chunk, end="", flush=True)

raw = rag_chain.invoke("민생지원금의 2차 지원금 지급시기와 지급액수와 관련된 내용을 'bullet points' 형식으로 정리해주세요")
clean = drop_think(raw)
print(clean)

- **지급시기**: 2025년 9월 22일(월)부터 2025년 10월 31일(금)까지  
- **지급액수**: 1인당 10만원 (전국민의 90% 대상)
